In [ ]:
%env LLM_API_KEY=替换为自己的Qwen API Key，打分用
%env LLM_BASE_URL=https://dashscope.aliyuncs.com/compatible-mode/v1

In [ ]:
%%capture --no-stderr
!pip install -U langchain langchain_community langchain_openai pypdf sentence_transformers chromadb shutil openpyxl FlagEmbedding

In [2]:
import langchain, langchain_community, pypdf, sentence_transformers, chromadb, langchain_core

for module in (langchain, langchain_core, langchain_community, pypdf, sentence_transformers, chromadb):
    print(f"{module.__name__:<30}{module.__version__}")

/opt/anaconda3/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


langchain                     0.2.10
langchain_core                0.2.28
langchain_community           0.2.9
pypdf                         4.3.1
sentence_transformers         3.0.1
chromadb                      0.5.4


In [3]:
import os
import pandas as pd

In [4]:
expr_version = 'retrieval_v9_parent_document_retriever'

preprocess_output_dir = os.path.join(os.path.pardir, 'outputs', 'v1_20240713')
expr_dir = os.path.join(os.path.pardir, 'experiments', expr_version)

# 读取文档

In [5]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(os.path.join(os.path.pardir, 'data', '2024全球经济金融展望报告.pdf'))
documents = loader.load()

qa_df = pd.read_excel(os.path.join(preprocess_output_dir, 'question_answer.xlsx'))

In [6]:
len(documents)

53

# 文档切分

## 现有切分方法

现有切分方法这部分不是必须的，但后续在评估检索性能时使用到了文档片段的uuid，为了能够评估检索效果，此处还是加入现有切分方法，通过关联知识片段的方法把uuid关联到ParentDocumentRetrieval，这样

In [10]:
from uuid import uuid4
import os
import pickle

def split_docs(documents, filepath, chunk_size=400, chunk_overlap=40, seperators=['\n\n\n', '\n\n'], force_split=False):
    if os.path.exists(filepath) and not force_split:
        print('found cache, restoring...')
        return pickle.load(open(filepath, 'rb'))

    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=seperators
    )
    split_docs = splitter.split_documents(documents)
    for chunk in split_docs:
        chunk.metadata['uuid'] = str(uuid4())

    pickle.dump(split_docs, open(filepath, 'wb'))

    return split_docs

In [11]:
splitted_docs = split_docs(documents, os.path.join(preprocess_output_dir, 'split_docs.pkl'), chunk_size=500, chunk_overlap=50)

found cache, restoring...


# 检索

## 准备检索器

In [12]:
import torch

from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.storage import InMemoryStore

model_path = 'BAAI/bge-large-zh-v1.5'

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'device: {device}')

def get_embeddings(model_path):
    embeddings = HuggingFaceBgeEmbeddings(
        model_name=model_path,
        model_kwargs={'device': device},
        encode_kwargs={'normalize_embeddings': True},
        query_instruction='为这个句子生成表示以用于检索相关文章：'
    )
    return embeddings

device: cuda


In [13]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.retrievers import ParentDocumentRetriever

parent_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=100,
    separators=['\n\n\n', '\n\n', '\n']
)
child_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    separators=['\n\n\n', '\n\n', '\n']
)

vectorstore = Chroma(
    collection_name='split_parents', embedding_function=get_embeddings(model_path)
)
store = InMemoryStore()

retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
    search_kwargs={'k': 3}
)

In [14]:
retriever.add_documents(documents)

In [15]:
len(vectorstore.similarity_search('2023年全球经济增长的特点是什么？')[0].page_content)

234

In [16]:
len(retriever.invoke('2023年全球经济增长的特点是什么？')[0].page_content)

679

# 预测

In [17]:
from langchain.llms import Ollama

ollama_llm = Ollama(
    model='qwen2:7b-instruct',
    base_url='http://localhost:11434'
)

In [18]:
def rag(question):
    prompt_tmpl = """
你是一个金融分析师，擅长根据所获取的信息片段，对问题进行分析和推理。
你的任务是根据所获取的信息片段（<<<<context>>><<<</context>>>之间的内容）回答问题。
回答保持简洁，不必重复问题，不要添加描述性解释和与答案无关的任何内容。
已知信息：
<<<<context>>>
{{knowledge}}
<<<</context>>>

问题：{{question}}
请回答：
""".strip()

    chunks = retriever.invoke(question)
    prompt = prompt_tmpl.replace('{{knowledge}}', '\n\n'.join([doc.page_content for doc in chunks])).replace('{{question}}', question)

    return ollama_llm.invoke(prompt), chunks

In [19]:
print(rag('2023年全球经济增长的特点是什么？')[0])

2023年全球经济增长的特点是“复苏+分化”，即全球经济在经历一段时间的调整后开始复苏，但不同地区、国家间的增长速度和表现存在显著差异。发达经济体增速明显放缓，而新兴经济体增长也面临挑战与机遇并存。

具体来看：

1. **发达国家**：
   - **美国经济**：未受加息明显冲击，出现超预期增长。服务领域的消费支出稳定增长，并且受益于就业市场稳健、劳动者实际收入增加和政府政策支持（如《通胀削减法案》和《芯片与科学法案》），推动了经济增长。
   - **欧元区和英国**：经济增长显著放缓，三季度GDP环比增速由正转负。

2. **新兴经济体**：
   - **东南亚地区**（例如菲律宾、印度尼西亚）：居民消费支出增长强劲，对GDP有较高拉动作用。经济增长率相对较高。
   - **中东地区**：经济增长依靠非能源领域，但由于高基数效应，增速明显减弱。沙特的GDP增长率降低至1.2%。
   - **拉美新兴经济体**（如巴西、墨西哥）：增长表现好于部分国家，GDP增速超过3%，但阿根廷面临通胀与负增长问题。
   - **非洲新兴经济体**：整体增长疲软，南非GDP同比增长为年内最高增速。各国通胀走势分化，南非通胀率相对较低，埃及和尼日利亚物价水平上涨。

全球贸易形势的好转有望带动东南亚经济体出口恢复，预计2024年全球经济增速将略高于2023年的4%，但仍低于前一年度的水平，约为2.5%。


In [20]:
from tqdm.auto import tqdm

prediction_df = qa_df[qa_df['dataset'] == 'test'][['uuid', 'question', 'qa_type', 'answer']].rename(columns={'answer': 'ref_answer'})

def predict(prediction_df):
    prediction_df = prediction_df.copy()
    answer_dict = {}

    for idx, row in tqdm(prediction_df.iterrows(), total=len(prediction_df)):
        uuid = row['uuid']
        question = row['question']
        answer, chunks = rag(question)

        answer_dict[question] = {
            'uuid': uuid,
            'ref_answer': row['ref_answer'],
            'gen_answer': answer,
            'chunks': chunks
        }
    prediction_df.loc[:, 'gen_answer'] = prediction_df['question'].apply(lambda q: answer_dict[q]['gen_answer'])
    prediction_df.loc[:, 'chunks'] = prediction_df['question'].apply(lambda q: answer_dict[q]['chunks'])

    return prediction_df

In [21]:
pred_df = predict(prediction_df)

  0%|          | 0/100 [00:00<?, ?it/s]

# 评估

In [22]:
from langchain_openai import ChatOpenAI
import time

judge_llm = ChatOpenAI(
    api_key=os.environ['LLM_API_KEY'],
    base_url=os.environ['LLM_BASE_URL'],
    model_name='qwen2-72b-instruct',
    temperature=0
)

def evaluate(prediction_df):
    """
    对预测结果进行打分
    :param prediction_df: 预测结果，需要包含问题，参考答案，生成的答案，列名分别为question, ref_answer, gen_answer
    :return 打分模型原始返回结果
    """
    prompt_tmpl = """
你是一个经济学博士，现在我有一系列问题，有一个助手已经对这些问题进行了回答，你需要参照参考答案，评价这个助手的回答是否正确，仅回复“是”或“否”即可，不要带其他描述性内容或无关信息。
问题：
<question>
{{question}}
</question>

参考答案：
<ref_answer>
{{ref_answer}}
</ref_answer>

助手回答：
<gen_answer>
{{gen_answer}}
</gen_answer>
请评价：
    """
    results = []

    for _, row in tqdm(prediction_df.iterrows(), total=len(prediction_df)):
        question = row['question']
        ref_answer = row['ref_answer']
        gen_answer = row['gen_answer']

        prompt = prompt_tmpl.replace('{{question}}', question).replace('{{ref_answer}}', str(ref_answer)).replace('{{gen_answer}}', gen_answer).strip()
        result = judge_llm.invoke(prompt).content
        results.append(result)

        time.sleep(1)
    return results

In [23]:
pred_df['raw_score'] = evaluate(pred_df)

  0%|          | 0/100 [00:00<?, ?it/s]

In [24]:
pred_df['raw_score'].unique()

array(['是', '否'], dtype=object)

In [25]:
pred_df['score'] = (pred_df['raw_score'] == '是').astype(int)

In [26]:
pred_df['score'].mean()

0.67

In [27]:
pred_df[['qa_type', 'score']].groupby('qa_type').mean().reset_index()

,qa_type,score
0,detailed,0.655914
1,large_context,0.857143


In [28]:
pred_df.to_excel(os.path.join(expr_dir, f'{expr_version}_prediction.xlsx'), index=False)